# 🚗 Análise e Otimização do Mapa VE

Este notebook implementa um sistema completo para análise de logs MSL do TunerStudio e otimização do mapa VE usando inteligência artificial.

## 📋 Fluxo de Trabalho

1. **Carregamento de Dados**: Leitura do arquivo .msl e tabelas VE/Lambda
2. **Análise de Qualidade**: Verificação da qualidade dos dados
3. **Estimativa de Delay**: Cálculo do delay da sonda O2
4. **Preparação para ML**: Processamento dos dados para machine learning
5. **Treinamento**: Treinamento do modelo de IA
6. **Otimização**: Geração do mapa VE otimizado
7. **Análise de Resultados**: Visualização e relatórios

## 1. Importações e Inicialização

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Importa nossas classes
from msl_data_processor import MSLDataProcessor
from ve_map_optimizer import VEMapOptimizer

# Configurações de plotagem
plt.style.use('default')
sns.set_palette("husl")

print("🚀 Ambiente configurado com sucesso!")
print("📊 Pronto para análise de dados MSL")

## 2. Carregamento de Dados

Carregamos os dados MSL e as tabelas VE/Lambda existentes.

In [ ]:
# Inicializa o otimizador
optimizer = VEMapOptimizer()

# Caminhos dos arquivos
msl_file = "logs/short.msl"  # Arquivo de log MSL
ve_table_file = "logs/veTable1Tbl_pre.table"  # Tabela VE atual
lambda_table_file = "logs/lambdaTable1Tbl_.table"  # Tabela Lambda Target

# Carrega todos os dados
success = optimizer.load_data(
    msl_file_path=msl_file,
    ve_table_path=ve_table_file, 
    lambda_table_path=lambda_table_file
)

if success:
    print("✅ Todos os dados carregados com sucesso!")
else:
    print("❌ Erro no carregamento dos dados")

## 3. Exploração Inicial dos Dados

Vamos examinar os dados carregados para entender sua estrutura.

In [ ]:
# Acessa os dados através do processador
log_data = optimizer.data_processor.log_data

# Mostra informações básicas
print("📊 INFORMAÇÕES DO LOG MSL:")
print(f"   Colunas disponíveis: {len(log_data.columns)}")
print(f"   Registros: {len(log_data)}")
print(f"   Duração: {log_data['Time'].max():.1f} segundos")

# Mostra as primeiras colunas
print("\n📋 Primeiras 10 colunas:")
for i, col in enumerate(log_data.columns[:10]):
    print(f"   {i+1:2d}. {col}")

# Mostra estatísticas dos dados principais
main_cols = ['RPM', 'MAP', 'Lambda', 'VE _Current', 'CLT']
available_cols = [col for col in main_cols if col in log_data.columns]

if available_cols:
    print("\n📈 ESTATÍSTICAS DOS DADOS PRINCIPAIS:")
    display(log_data[available_cols].describe())

In [ ]:
# Visualização da distribuição dos dados principais
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Distribuição dos Dados Principais', fontsize=16, fontweight='bold')

# RPM
if 'RPM' in log_data.columns:
    axes[0,0].hist(log_data['RPM'], bins=50, alpha=0.7, edgecolor='black')
    axes[0,0].set_title('Distribuição RPM')
    axes[0,0].set_xlabel('RPM')
    axes[0,0].set_ylabel('Frequência')

# MAP
if 'MAP' in log_data.columns:
    axes[0,1].hist(log_data['MAP'], bins=50, alpha=0.7, edgecolor='black')
    axes[0,1].set_title('Distribuição MAP (Pressão)')
    axes[0,1].set_xlabel('MAP (kPa)')
    axes[0,1].set_ylabel('Frequência')

# Lambda
if 'Lambda' in log_data.columns:
    axes[1,0].hist(log_data['Lambda'], bins=50, alpha=0.7, edgecolor='black')
    axes[1,0].set_title('Distribuição Lambda')
    axes[1,0].set_xlabel('Lambda')
    axes[1,0].set_ylabel('Frequência')
    axes[1,0].axvline(1.0, color='red', linestyle='--', label='Estequiométrico')
    axes[1,0].legend()

# VE Current
if 'VE _Current' in log_data.columns:
    axes[1,1].hist(log_data['VE _Current'], bins=50, alpha=0.7, edgecolor='black')
    axes[1,1].set_title('Distribuição VE Atual')
    axes[1,1].set_xlabel('VE (%)')
    axes[1,1].set_ylabel('Frequência')

plt.tight_layout()
plt.show()

## 4. Visualização das Tabelas VE e Lambda

Examinamos as tabelas carregadas.

In [ ]:
# Visualiza as tabelas carregadas
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Tabela VE
if optimizer.data_processor.ve_table is not None:
    im1 = axes[0].imshow(optimizer.data_processor.ve_table, aspect='auto', cmap='viridis')
    axes[0].set_title('Mapa VE Atual')
    axes[0].set_xlabel('RPM (bins)')
    axes[0].set_ylabel('FuelLoad (bins)')
    plt.colorbar(im1, ax=axes[0], label='VE (%)')
    
    # Adiciona valores dos eixos
    rpm_ticks = np.arange(0, len(optimizer.data_processor.rpm_bins_ve), 2)
    load_ticks = np.arange(0, len(optimizer.data_processor.load_bins_ve), 2)
    
    axes[0].set_xticks(rpm_ticks)
    axes[0].set_xticklabels([f"{int(optimizer.data_processor.rpm_bins_ve[i])}" for i in rpm_ticks])
    axes[0].set_yticks(load_ticks)
    axes[0].set_yticklabels([f"{int(optimizer.data_processor.load_bins_ve[i])}" for i in load_ticks])

# Tabela Lambda
if optimizer.data_processor.lambda_table is not None:
    im2 = axes[1].imshow(optimizer.data_processor.lambda_table, aspect='auto', cmap='coolwarm')
    axes[1].set_title('Mapa Lambda Target')
    axes[1].set_xlabel('RPM (bins)')
    axes[1].set_ylabel('FuelLoad (bins)')
    plt.colorbar(im2, ax=axes[1], label='Lambda Target')
    
    # Adiciona valores dos eixos
    rpm_ticks = np.arange(0, len(optimizer.data_processor.rpm_bins_lambda), 2)
    load_ticks = np.arange(0, len(optimizer.data_processor.load_bins_lambda), 2)
    
    axes[1].set_xticks(rpm_ticks)
    axes[1].set_xticklabels([f"{int(optimizer.data_processor.rpm_bins_lambda[i])}" for i in rpm_ticks])
    axes[1].set_yticks(load_ticks)
    axes[1].set_yticklabels([f"{int(optimizer.data_processor.load_bins_lambda[i])}" for i in load_ticks])

plt.tight_layout()
plt.show()

# Estatísticas das tabelas
print("📊 ESTATÍSTICAS DAS TABELAS:")
if optimizer.data_processor.ve_table is not None:
    ve_table = optimizer.data_processor.ve_table
    print(f"   VE Table: {ve_table.shape} - Range: {ve_table.min():.1f}% a {ve_table.max():.1f}%")

if optimizer.data_processor.lambda_table is not None:
    lambda_table = optimizer.data_processor.lambda_table
    print(f"   Lambda Table: {lambda_table.shape} - Range: {lambda_table.min():.3f} a {lambda_table.max():.3f}")

## 5. Estimativa do Delay da Sonda O2

Analisa eventos de aceleração para estimar o delay da sonda O2.

In [ ]:
# Estima o delay da sonda
estimated_delay = optimizer.estimate_sensor_delay()

print(f"⏱️ Delay estimado da sonda O2: {estimated_delay:.3f} segundos")

# Visualiza alguns eventos de TPS se disponível
if 'TPS' in log_data.columns and 'Lambda' in log_data.columns:
    # Calcula rate of change do TPS
    log_data_temp = log_data.copy()
    log_data_temp['TPS_Rate'] = log_data_temp['TPS'].diff() / log_data_temp['Time'].diff()
    
    # Encontra eventos rápidos
    rapid_events = log_data_temp[abs(log_data_temp['TPS_Rate']) > 20]
    
    if len(rapid_events) > 0:
        print(f"🔍 Encontrados {len(rapid_events)} eventos de TPS rápido para análise")
        
        # Plota os primeiros 3 eventos
        fig, axes = plt.subplots(min(3, len(rapid_events)), 1, figsize=(12, 8))
        if len(rapid_events) == 1:
            axes = [axes]
        
        for i, (idx, event) in enumerate(rapid_events.head(3).iterrows()):
            event_time = event['Time']
            
            # Janela de ±2 segundos
            window_mask = (log_data_temp['Time'] >= event_time - 1) & (log_data_temp['Time'] <= event_time + 3)
            window_data = log_data_temp[window_mask]
            
            if len(window_data) > 10:
                ax = axes[i] if len(rapid_events) > 1 else axes[0]
                
                # Plot TPS e Lambda
                ax2 = ax.twinx()
                
                ax.plot(window_data['Time'] - event_time, window_data['TPS'], 'b-', label='TPS (%)', linewidth=2)
                ax2.plot(window_data['Time'] - event_time, window_data['Lambda'], 'r-', label='Lambda', linewidth=2)
                
                ax.axvline(0, color='green', linestyle='--', alpha=0.7, label='Evento TPS')
                ax.axvline(estimated_delay, color='orange', linestyle='--', alpha=0.7, label=f'Delay estimado ({estimated_delay:.2f}s)')
                
                ax.set_xlabel('Tempo relativo (s)')
                ax.set_ylabel('TPS (%)', color='b')
                ax2.set_ylabel('Lambda', color='r')
                ax.set_title(f'Evento {i+1}: t={event_time:.1f}s')
                ax.legend(loc='upper left')
                ax2.legend(loc='upper right')
                ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    else:
        print("ℹ️  Nenhum evento de TPS rápido encontrado nos dados")
else:
    print("ℹ️  Dados de TPS não disponíveis para análise de delay")

## 6. Preparação dos Dados para Machine Learning

Processa os dados para treinar o modelo de IA.

In [ ]:
# Prepara os dados para treinamento
success = optimizer.prepare_training_data()

if success:
    print("✅ Dados preparados com sucesso!")
    
    # Mostra informações dos dados processados
    processed_data = optimizer.processed_data
    
    print(f"📊 Dataset de treinamento:")
    print(f"   Registros: {len(processed_data)}")
    print(f"   Features: {len(processed_data.columns) - 1}")
    
    # Mostra as features utilizadas
    feature_cols = [col for col in processed_data.columns if col != 'VE_Correction']
    print(f"\n🎯 Features utilizadas:")
    for i, col in enumerate(feature_cols, 1):
        print(f"   {i:2d}. {col}")
    
    # Estatísticas do target
    corrections = processed_data['VE_Correction']
    print(f"\n📈 Distribuição das correções VE:")
    print(f"   Média: {corrections.mean():.4f}")
    print(f"   Desvio: ±{corrections.std():.4f}")
    print(f"   Range: {corrections.min():.4f} a {corrections.max():.4f}")
    
    # Visualiza distribuição das correções
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Histograma das correções
    axes[0].hist(corrections, bins=50, alpha=0.7, edgecolor='black')
    axes[0].axvline(1.0, color='red', linestyle='--', label='Sem correção')
    axes[0].set_title('Distribuição das Correções VE')
    axes[0].set_xlabel('Fator de Correção')
    axes[0].set_ylabel('Frequência')
    axes[0].legend()
    
    # Scatter plot RPM vs Correção
    if 'RPM' in processed_data.columns:
        axes[1].scatter(processed_data['RPM'], corrections, alpha=0.5, s=1)
        axes[1].axhline(1.0, color='red', linestyle='--', label='Sem correção')
        axes[1].set_title('Correções por RPM')
        axes[1].set_xlabel('RPM')
        axes[1].set_ylabel('Fator de Correção')
        axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ Erro na preparação dos dados")

## 7. Treinamento do Modelo de Machine Learning

Treina o modelo para predição de correções VE.

In [ ]:
# Treina o modelo Random Forest
success = optimizer.train_model('random_forest')

if success:
    print("✅ Modelo treinado com sucesso!")
    
    # Analisa a importância das features
    if hasattr(optimizer.model, 'feature_importances_'):
        feature_cols = [col for col in optimizer.processed_data.columns if col != 'VE_Correction']
        importance_df = pd.DataFrame({
            'Feature': feature_cols,
            'Importance': optimizer.model.feature_importances_
        }).sort_values('Importance', ascending=False)
        
        # Plot da importância das features
        plt.figure(figsize=(10, 6))
        sns.barplot(data=importance_df, x='Importance', y='Feature')
        plt.title('Importância das Features no Modelo')
        plt.xlabel('Importância')
        plt.tight_layout()
        plt.show()
        
        print("\n🎯 Top 5 features mais importantes:")
        for i, (_, row) in enumerate(importance_df.head().iterrows(), 1):
            print(f"   {i}. {row['Feature']}: {row['Importance']:.3f}")
else:
    print("❌ Erro no treinamento do modelo")

## 8. Otimização do Mapa VE

Gera a nova tabela VE otimizada usando o modelo treinado.

In [ ]:
# Otimiza a tabela VE
optimized_table = optimizer.optimize_ve_table()

if optimized_table is not None:
    print("✅ Tabela VE otimizada gerada!")
    
    # Salva a tabela otimizada
    output_file = "ve_optimized.table"
    optimizer.save_optimized_table(optimized_table, output_file)
    
    print(f"💾 Tabela salva em: {output_file}")
else:
    print("❌ Erro na otimização da tabela")

## 9. Análise dos Resultados

Compara o mapa original com o otimizado e gera relatórios.

In [ ]:
if optimized_table is not None:
    # Gera gráficos de análise
    optimizer.generate_analysis_plots(optimized_table, "ve_analysis.png")
    
    # Gera relatório final
    optimizer.generate_report(optimized_table)

## 10. Comparação Detalhada

Análise mais detalhada das mudanças aplicadas.

In [ ]:
if optimized_table is not None:
    # Calcula diferenças
    original_table = optimizer.data_processor.ve_table
    differences = optimized_table - original_table
    percent_changes = (differences / original_table) * 100
    
    # Cria DataFrames para visualização
    rpm_bins = optimizer.data_processor.rpm_bins_ve
    load_bins = optimizer.data_processor.load_bins_ve
    
    original_df = pd.DataFrame(original_table, 
                              index=[f"Load_{int(l)}" for l in load_bins],
                              columns=[f"RPM_{int(r)}" for r in rpm_bins])
    
    optimized_df = pd.DataFrame(optimized_table,
                               index=[f"Load_{int(l)}" for l in load_bins], 
                               columns=[f"RPM_{int(r)}" for r in rpm_bins])
    
    changes_df = pd.DataFrame(percent_changes,
                             index=[f"Load_{int(l)}" for l in load_bins],
                             columns=[f"RPM_{int(r)}" for r in rpm_bins])
    
    # Mostra algumas estatísticas das mudanças
    print("📊 ANÁLISE DETALHADA DAS MUDANÇAS:")
    print(f"   Mudança média: {percent_changes.mean():.2f}%")
    print(f"   Desvio padrão: {percent_changes.std():.2f}%")
    print(f"   Maior aumento: {percent_changes.max():.2f}%")
    print(f"   Maior redução: {percent_changes.min():.2f}%")
    
    # Conta mudanças significativas por zona
    significant_mask = np.abs(percent_changes) > 2.0
    print(f"\n🎯 Mudanças significativas (>2%): {significant_mask.sum()} pontos")
    
    # Heatmap das mudanças percentuais
    plt.figure(figsize=(12, 8))
    sns.heatmap(changes_df, annot=True, fmt='.1f', cmap='RdBu_r', center=0,
                cbar_kws={'label': 'Mudança (%)'}, 
                xticklabels=[f"{int(r)}" for r in rpm_bins[::2]],
                yticklabels=[f"{int(l)}" for l in load_bins[::2]])
    plt.title('Mudanças Percentuais no Mapa VE (Otimizado - Original)')
    plt.xlabel('RPM')
    plt.ylabel('FuelLoad (kPa)')
    plt.tight_layout()
    plt.show()
    
    # Identifica as maiores mudanças
    flat_changes = percent_changes.flatten()
    flat_indices = np.unravel_index(np.argsort(np.abs(flat_changes))[-10:], percent_changes.shape)
    
    print("\n🔍 TOP 10 MAIORES MUDANÇAS:")
    for i in range(10):
        row_idx = flat_indices[0][-(i+1)]
        col_idx = flat_indices[1][-(i+1)]
        change = percent_changes[row_idx, col_idx]
        rpm = rpm_bins[col_idx]
        load = load_bins[row_idx]
        original_ve = original_table[row_idx, col_idx]
        new_ve = optimized_table[row_idx, col_idx]
        
        print(f"   {i+1:2d}. RPM {rpm:4.0f}, Load {load:2.0f}: {original_ve:.1f}% → {new_ve:.1f}% ({change:+.1f}%)")

## 🎯 Conclusões e Próximos Passos

### ✅ O que foi realizado:

1. **Carregamento de dados MSL**: Sistema agora lê corretamente os arquivos .msl do TunerStudio
2. **Análise de qualidade**: Verificação automática da qualidade dos dados
3. **Estimativa de delay**: Cálculo automático do delay da sonda O2
4. **Machine Learning**: Modelo treinado para predizer correções VE
5. **Otimização**: Geração de mapa VE otimizado
6. **Análise visual**: Gráficos e relatórios detalhados

### ⚠️ Avisos de Segurança:

- **Sempre monitore EGT** ao aplicar as mudanças
- **Teste em condições controladas** primeiro
- **Aplique mudanças incrementalmente** (25% por vez)
- **Mantenha backup** do tune original

### 🔄 Próximos passos recomendados:

1. **Validação em bancada**: Teste o mapa otimizado em dinamômetro
2. **Coleta de mais dados**: Logs em diferentes condições de operação
3. **Refinamento**: Iteração com dados de validação
4. **Monitoramento**: Acompanhamento de performance em tempo real